# TwoSix Armory Increment 2 Demo

Demonstrates the cross-model multi-factor analysis of TwoSix/Art Armory evaluations.
The dataset is [EuroSAT](https://github.com/phelber/EuroSAT),
a land use and land cover classification dataset. The three models in the comparison
are imported from HuggingFace's model hub: 

* nielsr/swin-tiny-patch4-window7-224-finetuned-eurosat
* nielsr/vit-finetuned-eurosat-kornia
* mrm8488/convnext-tiny-finetuned-eurosat

In [7]:
from pprint import pprint
import mlflow

"""retrieve the run id for the eurosat experiement"""

client = mlflow.tracking.MlflowClient("http://mlflow.gardproject.org:5000/")
experiment = client.get_experiment_by_name("eurosat-model-parameter-evaluation")

pprint(experiment)
pprint(dir(experiment))

results = client.get_run(experiment.experiment_id)

pprint(results)

<Experiment: artifact_location='s3://gard-mlflow-artifact-store/testing-mar-2023/14', creation_time=1694042447802, experiment_id='14', last_update_time=1694042447802, lifecycle_stage='active', name='eurosat-model-parameter-evaluation', tags={'mlflow.note.content': 'The goal of this file is to evaluate the EuroSAT '
                        'dataset using the top 3 fine-tuned models from HF and '
                        'measure the effect of epsilon. The evaluation is done '
                        'using Armory, and the results are uploaded to MLFlow. '
                        'Both the models and benign/adversarial images are '
                        'uploaded as artifacts.\n'
                        '\n'
                        'The dataset(s) are:\n'
                        '  - tanganke/EuroSAT\n'
                        '\n'
                        'The model(s) are:\n'
                        '  - '
                        'nielsr/swin-tiny-patch4-window7-224-finetuned-eurosat\n

AttributeError: 'MlflowClient' object has no attribute 'list_run_infos'